In [5]:
import pandas as pd
import spacy

from tqdm.notebook import tqdm
tqdm.pandas()



Ce code réalise une lemmatization à partir d'un modèle de machine learning issu de spacy.
installation du modèle : 
> python -m spacy download fr_core_news_sm

Ensuite on supprime certains mots fréquents, remis en forme et trouvés grâce à :
> from nltk.probability import FreqDist

Les nombres sont aussi filtrés.

J'ai réalisé cette approche car un  cleaning manuel était trop long (code en n² où n est le nombre de mots total du corpus). Le défaut de cette aproche est que potentiellement est elle trop bourrine et certains mots spécifiques qui nous intéressent ont disparu.

Il faut valider par une approche lexico et LDA les résultats => cf le code analyse_lexico.qmd

# Opening data

In [6]:
fichier = "../data/intermediate/base_merged.csv"

df = pd.read_csv(fichier, sep=';', quotechar='"')

df.sample(5)

,ID ASSO,ID ARTICLE,contenu,Type de document,Auteur,Date,Titre,URL
97,8,80001,« Dieu se rit des hommes qui se plaignent des ...,Article d'opinion,NaN,NaN,NaN,https://www.lactionfrancaise.fr/transidentite-...
296,13,130017,Kajsa Ekis Ekman note un virage à 180 degrés d...,blog,TRADFEM,26/09/2023,Kajsa Ekis Ekman note un virage à 180 degrés d...,https://tradfem.wordpress.com/2023/09/26/kajsa...
257,31,310007,Effrayant : un transgenre à l’apparence mascul...,Article d'opinion,NaN,2013-09-11 00:00:00,NaN,https://www.civitas-institut.com/2013/09/11/th...
320,13,130041,Entrevue de Kajsa Ekis Ekman à propos de son n...,blog,TRADFEM,06/08/2023,Entrevue de Kajsa Ekis Ekman à propos de son n...,https://tradfem.wordpress.com/2023/08/06/entre...
291,13,130012,Les transactivistes échouent à faire annuler u...,blog,TRADFEM,12/10/2023,Les transactivistes échouent à faire annuler u...,https://tradfem.wordpress.com/2023/10/12/les-t...


# Cleaning and tokenisation

In [7]:
# Load the French language model
nlp = spacy.load("fr_core_news_sm")

# Define your list of words to exclude
words_to_filter = [
    'le', 'la', 'les', 'de', 'un', 'à', 'être', 'suis', 'es', 'est', 'sont', 'été', 'étais', 'était', 'étaient', 'serai', 'seras', 'sera', 'seront',
    'et', 'en', 'avoir', 'ai', 'as', 'a', 'ont', 'eu', 'aurai', 'auras', 'aura', 'auront', 'avais', 'avait', 'avaient', 'que', 'pour',
    'dans', 'ce', 'il', 'qui', 'ne', 'en', 'sur', 'se', 'pas', 'plus',
    'par', 'je', 'avec', 'tout', 'faire', 'fais', 'fait', 'font', 'son', 'mettre', 'autre', 'on', 'mais',
    'nous', 'comme', 'ou', 'si', 'leur', 'y', 'dire', 'dis', 'dit', 'disent', 'elle', 'voir', 'devoir',
    'deux', 'même', 'pendant', 'aussi', 'vouloir', 'grand', 'mon', 'nouveau', 'aller',
    'venir', 'ceux', 'faire', 'quelque', 'trouver', 'donner', 'donne', 'donnes', 'donnent', 'aussi', 'autre',
    'tous', 'vieux', 'bon', 'voir', 'moins', 'trois', 'avant', 'sa', 'faire', 'contre',
    'abord', 'sous', 'ou', 'apporter', 'grand', 'ainsi', 'long', 'très', 'tout', 'avoir',
    'beau', 'chaque', 'peu', 'quoi', 'encore', 'aller', 'montrer', 'semaine', 'ainsi',
    'nuit', 'aussi', 'bien', 'deuxième', 'moins', 'tout', 'avoir', 'peu', 'nuit', 'ça',
    'mon', 'ma', 'mes', 'ton', 'ta', 'tes', 'son', 'sa', 'ses', 'notre', 'notre', 'nos', 'votre', 'votre', 'vos', 'leur', 'leur', 'leurs',
    'non', '’', '.', ',', ':', 'est', 'des', "l'", "d'", "une", "d", "»", "«", "du", "qu", "au", "n", "s", "elle", ")", "(", "aux", 'c', 'cette', 'un', 'ces','?', 'il', 'ils', 'j', "l", "-", "vous", "entre",
    "[", "]", "elles", '”', '“', "dont", "cela", ";", "m", "selon", "!", '–', "en", "autre", "ici", "ce", "ça", "cela", "ceci", "car", "ou", "où", "ni", "mais", "et", "donc", "parce", "que", "quand", "comment", "qui", "quoi",
    "à", "de", "pour", "sans", "sur"
]

# Function to tokenize, filter, and lemmatize text, excluding numbers
def tokenize_filter_and_lemmatize(text):
    # Process the text using spaCy
    doc = nlp(text)
    # Extract lemmatized forms of the words, excluding specific tokens and numbers
    lemmatized = " ".join([token.lemma_ for token in doc if token.lemma_ not in words_to_filter and not token.is_digit])
    return lemmatized

# Apply the function to the 'contenu' column
df['lemmatized_contenu'] = df['contenu'].progress_apply(tokenize_filter_and_lemmatize)

# Print the dataframe to see the result
df.sample(5)

  0%|          | 0/397 [00:00<?, ?it/s]

,ID ASSO,ID ARTICLE,contenu,Type de document,Auteur,Date,Titre,URL,lemmatized_contenu
363,13,130084,Prostitution…le monde à l’enversTRADFEM/04/06/...,blog,TRADFEM,04/06/2023,Prostitution…le monde à l’envers,https://tradfem.wordpress.com/2023/06/04/prost...,prostitution … monde l’ enverstradfem/04/06/20...
199,20,200062,"Lisa Selin Davis, auteur de TOMBOY : The Surpr...",Article d'opinion,NaN,NaN,NaN,https://cryforrecognition.be/fr/ce-qui-echappe...,Lisa Selin Davis auteur TOMBOY The Surprising ...
294,13,130015,Les transactivistes misogynes viennent de subi...,blog,TRADFEM,03/10/2023,Les transactivistes misogynes viennent de subi...,https://tradfem.wordpress.com/2023/10/03/les-m...,transactiviste misogyne subir énorme défaite R...
346,13,130067,Non au détournement du concept d’intersectionn...,blog,TRADFEM,23/06/2023,Non au détournement du concept d’intersectionn...,https://tradfem.wordpress.com/2023/06/23/non-a...,détournement concept d’ intersectionnalitétrad...
190,20,200053,Il y a aujourd’hui de plus en plus de jeunes –...,Article d'opinion,NaN,NaN,NaN,https://cryforrecognition.be/fr/est-il-justifi...,aujourd’hui jeune adolescent enfant ressentir ...


# Export

In [8]:
fichier = "../data/intermediate/base_lemmatized.csv"
df.to_csv(fichier, sep=';', index=False, quotechar='"')